In [2]:
from demo import train_test_demo
from demo import select_result_demo
from demo import input_result_demo
train_test_demo()

Dropdown(description='Dataset:', options=('Market1501', 'DukeMTMC'), value='Market1501')

Dropdown(description='Model:', options=('ResNet50', 'DenseNet121'), value='ResNet50')

Checkbox(value=True, description='Re_Ranking:')

Box(children=(Button(description='Train', style=ButtonStyle()), Button(description='Test', style=ButtonStyle())))

In [3]:
select_result_demo()

interactive(children=(IntSlider(value=5, description='R', max=20), IntSlider(value=0, description='index', max=3367), IntSlider(value=20, description='size', max=40), Output()), _dom_classes=('widget-interact',))

In [4]:
input_result_demo()

Button(description='Open Query Image', style=ButtonStyle())

<function matplotlib.pyplot.show>

interactive(children=(Text(value='0001_c1s1_001051_00.jpg', description='fname'), IntSlider(value=5, description='R', max=20), IntSlider(value=20, description='size', max=40), Output()), _dom_classes=('widget-interact',))